In [1]:
## dependencies
import os;from os import getcwd,chdir
path_vpsc_home = '/Users/yj/repo/vpsc/vpsc-dev-fld'
chdir(path_vpsc_home)
from fld import write_FLDnu
from MP import progress_bar
from MP.mat import mech
import tempfile, timeit, fld, subprocess, threading,time,shutil, multiprocessing
from multiprocessing import Process


upr = progress_bar.update_progress
uet = progress_bar.update_elapsed_time
FlowCurve = mech.FlowCurve
path_home = getcwd()


In [2]:
def dum(a,b,*wc):
    print a, b
    for w in wc:
        print w,
dum(3,1,3,3,4,5)

3 1
3 3 4 5


In [3]:
class workerFLDa():
    def __init__(self):
        self.path_home=os.getcwd()
        self.fc = FlowCurve()
        pass
    def create_working_folder(
            self,rho=0,cvm=1e-3,evm_limit=0.03,
            deij=1e-3,path_work=None):
        """
        Assign worker's duty
        The worker should create 'vpsc7.in' input
        according to the linear path (rho), strain rate (cvm),
        with an increment of <deij>. The worker should keep
        up the duty until <evm_limit> is reached.
        
        Arguments
        =========
        rho
        cvm
        evm_limit
        deij
        path_work
        """
        from fld import fld_pre_probe
        if path_work==None:
            self.path_work= tempfile.mkdtemp(dir='/tmp', 
                                             prefix='VPSC-FLDa-worker-')
        else: self.path_work = path_work
        print self.path_work, ' has been created'
        fld_pre_probe(rho, cvm, evm_limit, deij,
                      self.path_work,'FLD_nu.in','vpsc')
    def run(self,bufsize=1):
        """
        Execute VPSC and piping stdout
        
        Argments
        ========
        wait=False
        bufsize=1
        """
        os.chdir(self.path_work)
        ## piping to file object
        self.stdout = subprocess.PIPE
        self.fc.epsilon_vm=[]
        self.p = subprocess.Popen(['./vpsc'], 
                                shell=True,
                                stdout=self.stdout,
                                bufsize=bufsize)
        self.stdout=self.p.stdout
        self.pid=self.p.pid
        self.sigma = []
        self.epsilon = [] 
    def remove(self):
        shutil.rmtree(self.path_work)
        print 'path_work: %s has been removed'%self.path_work
        os.chdir(self.path_home)
        print 'Homing to %s'%self.path_home

#    def kill(self): self.p.kill()
#    def terminate(self): self.p.terminate()
#    def lock(self): self.p.wait()

    def readlines(self,nbuf=10,verbose=False):
        """
        Read <nbuf> number of lines from self.stdout
        
        Arguments
        =========
        nbuf=10
        verbose=False
        """
        for i in range(nbuf):
            line = self.stdout.readline().split('\n')[0]
            try:
                d=line.split('||')
                eps=np.array(map(float,d[0].split()))
                sig=np.array(map(float,d[1].split()))
            except: pass
            else:
                if verbose: print d
                self.sigma.append(sig)
                self.epsilon.append(eps)
                self.update_fc()
                self.fc.get_eqv()
    def set_monitor(self,term_evm=0.1,dt=5,nbuf=10):
        """
        Monitoring stress-strain, if p terminates
        prior to given term_evm, save/return that value.
        
        Arguments
        =========
        term_evm=0.1
        dt=5
        nbuf=10
        """
        t0=time.time()
        iexit=False
        while not(iexit):
            t1=time.time(); elps = t1-t0
            uet(elps,False,'Total elapsed time')
            self.readlines(nbuf=nbuf)
            try: 
                dum=self.fc.nstp
            except: pass
            else:
                #print self.fc.epsilon_vm[-1], term_evm
                if self.fc.epsilon_vm[-1]>=term_evm:
                    #self.p.kill()
                    self.p.terminate()
                    print '\nprocess killed due to exceeding term_evm'
                    return self.fc.epsilon_vm[-1]
                    iexit = True
            if self.p.poll()!=None:
                print 'process terminated'
                print self.p.poll();iexit=True
                if self.fc.epsilon_vm[-1]<term_evm:
                    print 'VM strain found from pid:%i '%self.p.pid,
                    print 'is lower than the given value!'
                    return self.fc.epsilon_vm[-1]
            if not(iexit): time.sleep(dt)
    def update_fc(self):
        """
        Update flow stress/strain data
        """
        eps,sig = np.array(self.epsilon).T, np.array(self.sigma).T
        self.fc.get_6strain(eps)
        self.fc.get_6stress(sig)
    def returncode(self):
        return p.returncode

In [4]:
"""
Workers form a union.
"""
class WorkUnion:
    def __init__(self):
        pass
    def assembly(self):
        pass
    def set_monitor(self):
        pass

In [5]:
pwd

u'/Users/yj/repo/vpsc/vpsc-dev-fld'

In [6]:
ls *.in

Cubcomp.in*  FLD_nu.in    MANIFEST.in  vpsc7.in


In [7]:
myFLDa=workerFLDa()
myFLDa.create_working_folder(evm_limit=0.01)
print myFLDa.path_work

/tmp/VPSC-FLDa-worker-Gsa24a  has been created


TypeError: fld_pre_probe() takes at most 6 arguments (7 given)

In [8]:
myFLDa.run()
print myFLDa.pid
myFLDa.readlines()

72141


In [9]:
myFLDa.readlines()

In [10]:
myFLDa.set_monitor(dt=0.5,term_evm=0.01)

Total elapsed time: 00 [sec]
process killed due to exceeding term_evm


0.01268219641378706

In [11]:
myFLDa.p.kill()

In [12]:
myFLDa.remove()

path_work: /tmp/VPSC-FLDa-worker-9O8R2M has been removed
Homing to /Users/yj/repo/vpsc/vpsc-dev-fld


In [13]:
myFLDa.readlines(nbuf=5)

In [14]:
myFLDa.fc.epsilon_vm[-2]

0.017293614440338348

In [15]:
print myFLDa.fc.epsilon
print myFLDa.fc.sigma

[[[ 0.001  0.002  0.003  0.004  0.005  0.006  0.007  0.008  0.009  0.01
    0.011  0.012  0.013  0.014  0.015  0.016]
  [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]]

 [[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]]

 [[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]
  [ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   ]
  [-0.001 -0.002 -0.003 -0.004 -0.005 -0.006 -0.007 -0.008 -0.009 

In [16]:
%whos

Variable          Type                          Data/Info
---------------------------------------------------------
FlowCurve         classobj                      MP.mat.mech.FlowCurve
Process           type                          <class 'multiprocessing.process.Process'>
WorkUnion         classobj                      __main__.WorkUnion
chdir             builtin_function_or_method    <built-in function chdir>
dum               function                      <function dum at 0x1063ce1b8>
fld               module                        <module 'fld' from 'fld.py'>
getcwd            builtin_function_or_method    <built-in function getcwd>
mech              module                        <module 'MP.mat.mech' fro<...>ackages/MP/mat/mech.pyc'>
multiprocessing   module                        <module 'multiprocessing'<...>processing/__init__.pyc'>
myFLDa            __main__.workerFLDa           <__main__.workerFLDa instance at 0x1055e4200>
os                module                        <mo

# Forming limit diagram prediction using VPSC for region B

In [17]:
ls *.tar

VPSCFLDb-vpscin-534FkX.tar     VPSCFLDb-vpscin-KwvDvn.tar     fld_a_rst_00.tar               fld_a_rst_01.tar               fld_a_rst_02.tar               fld_a_rst_20150305_JyjeTz.tar


In [18]:
import os

path_vpsc_home = '/Users/yj/repo/vpsc/vpsc-dev-fld'
fns=os.listdir(path_vpsc_home)
tar_fn_regiona='/Users/yj/repo/vpsc/vpsc-dev-fld/fld_a_rst_20150305_JyjeTz.tar'
p=os.popen('tar -tf %s'%tar_fn_regiona)
tar_fn_regiona_paths=p.readlines()
for i in range(len(tar_fn_regiona_paths)):
    tar_fn_regiona_paths[i]=tar_fn_regiona_paths[i].split('\n')[0]
print tar_fn_regiona_paths

class NM_list:
    def __init__(self,n,m):
        self.container=[]
        for i in range(n):
            self.container.append([])
            for j in range(m):
                self.container[i].append(None)

['fld_a_rst_00.tar', 'fld_a_rst_01.tar', 'fld_a_rst_02.tar', 'fld_a_rst_03.tar']


In [19]:
d=NM_list(5,3)
print d.container

[[None, None, None], [None, None, None], [None, None, None], [None, None, None], [None, None, None]]


## Tasks for FLDb

In [20]:
ps=[]

In [21]:
"""
reference: http://pymotw.com/2/multiprocessing/communication.html
"""

path_vpsc_home = '/Users/yj/repo/vpsc/vpsc-dev-fld'
os.chdir(path_vpsc_home)
path_home = os.getcwd()

print 'path home:',path_home

class FLDbTask(object):
    def __init__(self,path_work=None,f0=0.990,psi0=0.,fn_tar_FLDa=None,iprobe=0):
        """
        """
        self.path_home=os.getcwd()
        ## create working folder
        self.create_working_folder(path_work,f0,psi0,fn_tar_FLDa,iprobe)
        pass
    def create_working_folder(
            self,path_work=None,f0=0.990,psi0=0.,
            fn_tar_FLDa=None,iprobe=0):
        """
        Assign worker's duties
        The worker should create 'vpsc7.in' input
        according to the linear paths given to him/her.
        
        Arguments
        =========
        path_work   = None
        f0          = 0.990
        psi0        = 0.
        fn_tar_FLDa = None
        iprobe      = 0    (according to the order of region_a_??.tar)
        """
        from fld import fld_post_probe2
        if not(os.path.isfile(fn_tar_FLDa)):
            raise IOError, 'Failed to find %s'%fn_tar_FLDa
        self.tf_a=fn_tar_FLDa


        self.path_temp = tempfile.mkdtemp(dir='/tmp',prefix='VPSCFLD-tarx-')
        memb  = 'fld_a_rst_%2.2i.tar'%iprobe
        cmd   = 'tar -C %s -xf %s %s'%(self.path_temp, self.tf_a, memb)
        iflag = os.system(cmd)
        if iflag!=0: raise IOError
        cmd   = 'tar -C %s -xf %s region_a.bin'%(self.path_temp, os.path.join(self.path_temp,memb))
        iflag = os.system(cmd) 
        if iflag!=0: raise IOError, 'Could not find region_a.bin'

        if path_work==None:
            self.path_work= tempfile.mkdtemp(
                dir='/tmp', prefix='VPSCFLDb-f%5.3f-r%2.2i-p%2.2i-'%(f0,iprobe,psi0))
        else: self.path_work = path_work
        shutil.move(os.path.join(self.path_temp,'region_a.bin'), self.path_work)
        shutil.rmtree(self.path_temp)
        write_FLDnu(dst=self.path_work,
                        limit_factor=10,
                        njacob_cal=1,
                        dx_jac=1e-04,
                        err_fi=1e-01,
                        err_vi=1e-07,
                        max_iter=20)
        #               f0,psi0,src_path,      dst,          *fns
        fld_post_probe2(f0,psi0,path_vpsc_home,self.path_work,
                        os.path.join(path_vpsc_home,'vpsc'),
                        os.path.join(path_vpsc_home,'find_rhos')
                        )
        os.chdir(path_home)
        
    def run(self,bufsize=10,rt=False):
        """
        Execute VPSC and piping stdout        
        write stdout stream to subprocess.PIPE

        Arguments
        ========
        bufsize = 10 (bufsize for 'vpsc' stdout through subprocess.Popen)
        rt      = False (realtime monitor flag)
                  if True, create, self.p_rt that runs the wrapper for realtime_monitor
        """
        import multiprocessing as mp
        ## change dir to self.path_work
        os.chdir(self.path_work)
        ## Construct pipe lines and relevant file objects
        self.stdout_fo = open('stdout.out','w') ## stdout file object
        self.stderr = open('stderr.out','w')
        ## initialize 'shared' lists that contain stress and strain.
        self.sigmaa = []; self.sigmab = []
        self.epsilona = []; self.epsilonb = []
        self.psi = []; self.f = []
        ## create a pipe line
        self.p = subprocess.Popen(['./vpsc'],stdout=subprocess.PIPE,
                                  stderr=self.stderr,bufsize=bufsize)
        self.pid = self.p.pid
        print 'pid:', self.pid
        self.stdout = self.p.stdout
        
        self.wrap_rt_monit(dt=0.1)
        ## running wrap_rt_monit as a subprocess?
        ## ---- MyTask.wrap_rt_monit(dt=0.1)
        #self.p_monit = mp.Process(target=self.wrap_rt_monit)
        #self.p_monit.start()
        #self.p_monit.join()
        os.chdir(self.path_home)
        
    def wrap_rt_monit(self,term_e1=1.,monit_timeout=36000,dt=0.1):
        """
        Wrapper for real time monitor
        
        term_e1=1.
        dt=0.1
        monit_timeout = 36000 (timeout for wrap_rt_monit per ce)
        """
        t0=time.time();exit=False
        while not(exit):
            self.realtime_monitor(term_e1=term_e1,
                                  dt=dt,nbuf=10,timeout=None)
            if time.time()-t0>monit_timeout:
                print 'Wraning: Timeout in self.wrap_rt_monit excedding ',\
                        monit_timeout
                exit=True
            if not(exit):
                if self.p.poll()!=None: exit=True
                    

    def kill(self): self.p.kill
    def terminate(self): self.p.terminate()
    def rm(self): shutil.rmtree(self.path_work)
    def remove(self): self.rm()
    def readlines(self,nbuf=10,verbose=False,f_crit=10.):
        """
        Read <nbuf> number of lines from self.stdout
        
        Arguments
        =========
        nbuf=10.
        verbose=False
        f_crit=10.
        """
        for i in range(nbuf):
            rl=self.stdout.readline()
            #print rl
            self.stdout_fo.write(rl)
            line = rl.split('\n')[0]

            try:
                a,b=line.split('||')
            except:
                """
                1) Either head streams from VPSC
                2) or streams dealing with vi-NR iteration
                """
                d=line.split()                
                pass
            
            else:
                adat, bdat=a.split(), b.split()
                f = float(bdat[9])
                if f>f_crit and abs(f)>abs(f_crit*1.5):
                    print 'Fail [2]'
                    e11_a, e22_a, e12_a, s11_a, s22_a, s12_a, \
                    e11_b, e22_b, e12_b, s11_b, s22_b, s22_b, psi=\
                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,\
                    np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
                    ifail=True
                    self.stdout_fo.close()
                else:
                    e11_a, e22_a, e12_a, s11_a, s22_a, s12_a = \
                    map(float,[adat[1],adat[2],adat[4],adat[5],adat[6],adat[8]])
                    e11_b, e22_b, e12_b, s11_b, s22_b, s12_b = \
                    map(float,[bdat[0],bdat[1],bdat[3],bdat[4],bdat[5],bdat[7]])
                    psi = float(bdat[10])
                    
                    #print e11_a,e22_a,e12_a,s11_a,s22_a,s12_a,f,psi

                #if verbose:
                #    #print e11_a, e22_a,e12_a, e11_b,e22_b,e12_b
                self.epsilona.append([e11_a,e22_a,-e11_a-e22_a,0,0,e12_a])
                self.epsilonb.append([e11_b,e22_b,-e11_b-e22_b,0,0,e12_b])
                self.sigmaa.append([s11_a,s22_a,0,0,0,s12_a])
                self.sigmab.append([s11_b,s22_b,0,0,0,s12_b])
                self.psi.append(psi)
                self.f.append(f)

    def realtime_monitor(self,term_e1=0.1,dt=1,nbuf=10,timeout=None):
        """
        Monitoring stress-strain, if p terminates
        prior to given term_e1 exit right off.

        Arguments
        =========
        term_evm = 0.1
        dt       = 5
        nbuf     = 10
        timeout  = None
        """
        t0=time.time()
        iexit=False
        while not(iexit):
            t1=time.time(); elps = t1-t0
            uet(elps,False,'Total elapsed time')
            ## risk that self.readlines may not work properly
            self.readlines(nbuf=nbuf)
            try: 
                dum=self.fc.nstp
            except: pass
            else:
                try: 
                    e11_a = self.epsilona[-1][0]
                except:pass
                else:
                    if e11_a>=term_e1:
                        self.p.kill()
                        iexit = True
            if self.p.poll()!=None:
                iexit=True
                self.stdout_fo.close()
                pass
            if timeout!=None: 
                #print 'Warning: Time-out in realtime_monitor'
                if time.time()-t0>timeout: iexit=True
            if not(iexit): time.sleep(dt)

    def update_fc(self):
        """
        Update flow stress/strain data
        """
        epsa,siga = np.array(self.epsilona).T, np.array(self.sigmaa).T
        self.fc_a = FlowCurve()
        self.fc_a.get_6strain(epsa)
        self.fc_a.get_6stress(siga)
        
        epsb,sigb = np.array(self.epsilonb).T, np.array(self.sigmab).T
        self.fc_b = FlowCurve()
        self.fc_b.get_6strain(epsb)
        self.fc_b.get_6stress(sigb)
        
    def returncode(self):
        return p.returncode

path home: /Users/yj/repo/vpsc/vpsc-dev-fld


In [22]:
ls *.tar

VPSCFLDb-vpscin-534FkX.tar     VPSCFLDb-vpscin-KwvDvn.tar     fld_a_rst_00.tar               fld_a_rst_01.tar               fld_a_rst_02.tar               fld_a_rst_20150305_JyjeTz.tar


## Serial calulcation example

In [23]:
os.chdir(path_vpsc_home)
MyTask=FLDbTask(path_work=None,f0=0.990,psi0=5.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=1)

making up VPSC7.in in: /tmp/VPSCFLDb-f0.990-r01-p05-XjxVNr/vpsc7.in


In [24]:
os.chdir(path_vpsc_home)
MyTask=FLDbTask(path_work=None,f0=0.990,psi0=5.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=1)
## MyTask run starts
MyTask.run() ## reading stdout is inluced in self.run
print '\n',MyTask.epsilona[-1::]
MyTask.rm()

## serial
MyTask=FLDbTask(path_work=None,f0=0.990,psi0=15.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=2)
## MyTask run starts
MyTask.run() ## reading stdout is inluced in self.run
print '\n',MyTask.epsilona[-1::]
MyTask.rm()


making up VPSC7.in in: /tmp/VPSCFLDb-f0.990-r01-p05-df1fch/vpsc7.in
pid: 72238
Total elapsed time: 25 [sec]
[[0.338, 0.0, -0.338, 0, 0, 0.0]]
making up VPSC7.in in: /tmp/VPSCFLDb-f0.990-r02-p15-8Ocuhx/vpsc7.in
pid: 72294
Total elapsed time: 25 [sec]
[[0.371, 0.1855, -0.5565, 0, 0, 0.0]]


# Based on multiprocessing Queue

In [25]:
os.chdir(path_vpsc_home)
def worker(q):
    obj=q.get()
    obj.run()
    
    
queue=multiprocessing.Queue()

p=multiprocessing.Process(target=worker,args=(queue,))
p.start()
queue.put(FLDbTask(path_work=None,f0=0.990,psi0=20.,
                   fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',
                   iprobe=2))

# wait for the worker to finish
queue.close()
queue.join_thread()
p.join()

making up VPSC7.in in: /tmp/VPSCFLDb-f0.990-r02-p20-kIbhCR/vpsc7.in
pid: 72373
Total elapsed time: 27 [sec]

# Joinable Que

In [53]:
import multiprocessing as mp
logger = mp.log_to_stderr(logging.INFO)

os.chdir(path_vpsc_home)

class Consumer(multiprocessing.Process):
    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue

    def run(self):
        proc_name = self.name
        while True:
            next_task = self.task_queue.get()
            if next_task is None:
                # Poison pill means shutdown
                print '%s: Exiting' % proc_name
                self.task_queue.task_done()
                break
            print '%s: %s' % (proc_name, next_task)
            answer = next_task()
            self.task_queue.task_done()
            self.result_queue.put(answer)
        return

## FLDb tasks           
class Task(object):
    def __init__(self,path_work=None,f0=0.990,psi0=5.,
                 fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=2):
        self.path_work=None
        self.f0=f0
        self.psi0=psi0
        self.fn_tar_FLDa=fn_tar_FLDa
        self.iprobe=iprobe
#        pass
#    def __call__(self):
        self.MyTask=FLDbTask(self.path_work,self.f0,self.psi0,self.fn_tar_FLDa,self.iprobe)
        ## MyTask run starts
        self.MyTask.run() ## reading stdout is inluced in self.run
        return self.MyTask.epsilona[-1]
    def __str__(self):
        return "f: %5.3f  psi0: %2.2i   iprobe: %2.2i"%(self.f0,self.psi0,self.iprobe)

# Establish communication queues
tasks = multiprocessing.JoinableQueue()
results = multiprocessing.Queue()
    
# Start consumers
num_consumers = 2 #multiprocessing.cpu_count() * 2
print 'Creating %d consumers' % num_consumers
consumers = [Consumer(tasks, results)
              for i in xrange(num_consumers)]
print 'Consumers created'
for w in consumers:
    print 'Started:', w
    w.start()

# Enqueue jobs
num_jobs = 2
all_tasks=[]
for i in xrange(num_jobs):
    #tasks.put(Task(path_work=None,f0=0.990,psi0=0.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=i))
    t=Task(path_work=None,f0=0.990,psi0=0.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=i)
    all_tasks.append(t)
    tasks.put(t)
    
# Add a poison pill for each consumer
for i in xrange(num_consumers):
    tasks.put(None)

#t0=time.time();exit=False
#while not(exit):
#    for 

## Wait for all of the tasks to finish
#tasks.join()
    
## Start printing results
#while num_jobs:
#    result = results.get()
#    print 'Result:', result
#    num_jobs -= 1
    

NameError: name 'logging' is not defined

In [51]:
help(all_tasks[0])

Help on Task in module __main__ object:

class Task(__builtin__.object)
 |  Methods defined here:
 |  
 |  __call__(self)
 |  
 |  __init__(self, path_work=None, f0=0.99, psi0=5.0, fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar', iprobe=2)
 |  
 |  __str__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __slotnames__ = []



Total elapsed time: 34 [sec]

Total elapsed time: 10 [sec]

Total elapsed time: 10 [sec]

# Check a job and monitor the stdout real-time.

In [34]:
os.chdir(path_vpsc_home)
## Testing FLDbTask()
MyTask=FLDbTask(path_work=None,f0=0.990,psi0=5.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=2)
## MyTask run starts
MyTask.run()
#print MyTask.p_monit
print '# MyTask Path Work:',MyTask.path_work
print '# MyTask pid:', MyTask.pid
ps.append(MyTask.p)

MyTask.wrap_rt_monit(dt=0.1)

making up VPSC7.in in: /tmp/VPSC-test-Kb1Fo8/vpsc7.in
pid: 19778
Total elapsed time: 17 [sec]

KeyboardInterrupt: 

In [294]:
MyTask.epsilona[-2:]

[[0.354, 0.177, -0.5309999999999999, 0, 0, 0.0],
 [0.355, 0.1775, -0.5325, 0, 0, 0.0]]

In [295]:
#MyTask.epsilona[-1]
#MyTask.p.kill()
MyTask.rm()

# Check a job and submit monitoring as an independent multiprocessing.process

In [296]:
os.chdir(path_vpsc_home)
## Testing FLDbTask()
MyTask=FLDbTask(path_work=None,f0=0.990,psi0=5.,fn_tar_FLDa='fld_a_rst_20150305_JyjeTz.tar',iprobe=2)

## MyTask run starts
MyTask.run()
#print MyTask.p_monit
print '# MyTask Path Work:',MyTask.path_work
print '# MyTask pid:',MyTask.pid
ps.append(MyTask.p)
#MyTask.realtime_monitor(timeout=5,nbuf=20,dt=0.1)
#p_monit= Process(target=MyTask.wrap_rt_monit,)

import multiprocessing; reload(multiprocessing)
p = multiprocessing.Process(target=MyTask.wrap_rt_monit)
p.start()
p.join()

making up VPSC7.in in: /tmp/VPSC-test-jAXcs2/vpsc7.in
pid: 18409
# MyTask Path Work: /tmp/VPSC-test-jAXcs2
# MyTask pid: 18409
Total elapsed time: 00 [sec]

In [297]:
p

<Process(Process-22, stopped)>

In [298]:
MyTask.epsilona[-2::]

[]

In [248]:
print p

<Process(Process-12, stopped)>


In [151]:
MyTask.realtime_monitor(timeout=5,nbuf=20,dt=0.1)

Total elapsed time: 04 [sec]

KeyboardInterrupt: 

In [ ]:
MyTask.epsilona

In [ ]:
MyTask.kill()
for i in range(len(ps)):
    ps[i].kill()

In [152]:
MyTask.p.kill()

In [360]:
os.chdir(path_vpsc_home)
class TaskManager:
    """
    Task belonging to the same paths with various angles.
    """
    def __init__(self):
        self.tasks=[]
        pass
    def givetask(self,*tasks):
        for task in tasks:
            self.tasks.append(task)
    def run(self):
        for task in self.tasks:
            task.run()
    def killall(self):
        for task in self.tasks:
            task.kill()
    def lock_until(self,buf_time=5.):
        iexit=False
        t0=time.time()
        while not(iexit):
            ## if any
            for i in range(len(self.tasks)):
                t = self.tasks[i]
                #print '\nPoll:', t.p.poll()
                if t.p.poll()!=None:
                    iexit=True
                    break
            t1=time.time()
            uet(t1-t0)
            if not(iexit): time.sleep(buf_time)

        ## find that 'successfully' ends first..
        #   find those which successfully read the first lines
        
## example with two angles
Task1=FLDbTask()
Task1.create_working_folder(
path_work=None,f0=0.990,psi0=0.,
fn_tar_FLDa='fld_a_rst_20150301_9wCeUP.tar',iprobe=0)

Task2=FLDbTask()
Task2.create_working_folder(
path_work=None,f0=0.990,psi0=5.,
fn_tar_FLDa='fld_a_rst_20150301_9wCeUP.tar',iprobe=0)

print Task2.path_work

TaskMan=TaskManager()
TaskMan.givetask(Task1, Task2)

TaskMan.run()
#TaskMan.lock_until(buf_time=5.)



/tmp/VPSC-test-Nph3FC
pid: 66726
pid: 66727


AttributeError: 'file' object has no attribute 'returncode'

In [349]:
MyTask.readlines(nbuf=10,timeout=10)

read lines exceeded nbuf


In [347]:
MyTask.p.kill()
shutil.rmtree(MyTask.path_work)
os.chdir(path_home)

## Manager of FLDb workers

In [283]:
import fld
from testing import datetime
from fld import fld_post_probe
import shutil
from random import choice
import numpy as np
from tempfile import mkdtemp

import multiprocessing

       
class Worker:
    def __init__(self):
        self.idle=True
        self.task=None
    def Give_task(self,task):
        os.getcwd(dir)
        self.task=task
    def KillTask(self):
        task.kill()
        self.idle=True
        self.task=None
        
        
class WorkerPool:  ## Pool of workers
    def __init__(self,nworker):
        self.workers=[]
        for i in range(nworker):
            self.workers.append(Worker())
        self.nworker=nworker
    def Assign_a_task(self,task,time_buffer=1.):
        """
        Assign the given task to an idle worker
        
        Arguments
        =========
        task
        time_buffer=1.
        """
        while (self.n_idle_workers==0):
            time.sleep(time_buffer)
            self.Find_idle_workers()
        
        chosen_worker = choice(self.idle_workers)
        chosen_worker.Give_Task(task)
    def Find_idle_workers(self):
        """
        See if any idle worker is available
        """
        self.n_idle_workers=0
        self.idle_workers=[]
        for i in range(len(self.workers)):
            if self.workers.idle:
                self.n_idle_workers=self.n_idle_workers+1
                self.idle_workers.append(self.workers[i])
    
class FLDmanager:
    def __init__(self):
        pass
    def gen_workers_union(self,nworker=1):
        self.WorkerPool = WorkerPool(nworker=nworker)
        self.nworker=self.WorkerPool.nworker
        pass
    def create_work_place(
        self,dir='/Users/yj/repo/vpsc/vpsc-dev-fld',
        fn='fld_a_rst_20150226.tar',
        f0=0.990,npsi=2):
        """
        1. Point to home and find
        what today's duties are.
        2. Create hub 
        3. Figure out what tasks are given
        4. Prepare tasks and assign them to a finite number of workers
        5. Make sure workers are not idle.
        
        Arguments
        =========
        dir
        fn   : tar file generated by region A simulations for <n> number of paths
        f0   : initial inhomogeneity factor (ratio of thicknesses)
        npsi=5.
        """
        if not(os.path.isfile(os.path.join(dir,fn))):
            from glob import glob
            print '-'*20
            print "could not find %s"%fn
            print 'Consider one of below listed file(s)'            
            fns=glob('fld_a_???*.tar')
            for i in range(len(fns)):
                print '%2.2i'%(i+1), fns[i]
            print '-'*20

        self.path_home = dir
        self.tf_a = fn
        self.psi0s = np.linspace(0.,90.,npsi)
        self.npsi = npsi
        
        ## generate path_hub
        self.path_hub = mkdtemp(dir='/tmp',prefix='VPSCFLDb-manager-hub-regionB-')
        os.chdir(self.path_home)
        fld_post_probe(f0=f0,psi0=0,# psi0 is not meaningful for path_hub
                       dst=self.path_hub,wc=['find_rhos','FLD_nu.in',self.tf_a])
        os.chdir(self.path_hub)
        p=os.popen('tar -tf '+self.tf_a)
        self.n_probe=len(p.readlines())
        print 'n_probe:', self.n_probe
        
        ## total tasks: [self.npsi, self.n_probe]
        ## Create the table of tasks.
        
        self.paths_work=NM_list(self.npsi,self.n_probe).container
        self.ntask = self.n_probe*self.npsi
        k=0
        print '# Prep total %i working paths'%self.ntask
        t0=time.time();dt=0.1
        self.tasks=[]
        tt0=t0
        for j in range(self.n_probe):
            for i in range(self.npsi):
                path_work = mkdtemp(dir='/tmp',prefix='VPSCFLDb-%2.2i-%2.2i-'%(j,self.psi0s[i]))
                self.paths_work[i][j] = path_work
                
                ## create tasks
                task = FLDbTask()
                task.create_working_folder(
                    path_work=path_work,f0=f0,
                    psi0=self.psi0s[i],fn_tar_FLDa=self.tf_a,
                    iprobe=j)
                self.tasks.append(task)      
                
                ## progress bar
                k=k+1
                t1=time.time()
                if t1-t0>dt or k==self.ntask:                    
                    upr(float(k)/float(self.ntask))
                    t0 = t1
        uet(time.time()-tt0);print
        os.chdir(self.path_home)
        
    def remove_all_workplaces(self):
        """
        Remove all workplaces, caution should be taken!
        """
        for i in xrange(len(self.paths_work)):
            for j in xrange(len(self.paths_work[i])):
                shutil.rmtree(self.paths_work[i][j])
        if self.path_hub!=None: shutil.rmtree(self.path_hub)

    def run(self,ncpu=None):
        """
        Arguments
        =========
        ncpu=None
        """
        if ncpu==None:
            self.ncpu=multiprocessing.cpu_count
        else: self.ncpu=ncpu
        print '\n'
        print '-'*41
        print ' VPSC-FLDb simulations for',
        print 'total %i runs'%(self.ntask)
        print ' Worker pool: %i'%self.nworker
        print '-'*41
        
        
        

In [284]:
evil_manager=FLDmanager()
evil_manager.create_work_place(npsi=3,dir='/Users/yj/repo/vpsc/vpsc-dev-fld',
        fn='fld_a_rst_20150301_9wCeUP.tar')
evil_manager.gen_workers_union(nworker=4)
evil_manager.run()
evil_manager.remove_all_workplaces()

n_probe: 4
# Prep total 12 working paths
Percent: [##############################] 100% Done...
Elapsed time: 00 [sec]


-----------------------------------------
 VPSC-FLDb simulations for total 12 runs
 Worker pool: 4
-----------------------------------------


In [282]:
evil_manager.remove_all_workplaces()

OSError: [Errno 2] No such file or directory: '/tmp/VPSCFLDb-00-00-PFk_5n'